In [3]:
%%pyspark

df = spark.read.load('abfss://capture@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/Integrated_Library_System__ILS__Data_Dictionary.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(synasp1, 8, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 833d24a8-8bc8-46ea-8546-ea00f5837f9c)

In [4]:
%%pyspark

# Show Schema
df.printSchema()

StatementMeta(synasp1, 8, 4, Finished, Available)

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Code Type: string (nullable = true)
 |-- Format Group: string (nullable = true)
 |-- Format Subgroup: string (nullable = true)
 |-- Category Group: string (nullable = true)
 |-- Category Subgroup: string (nullable = true)
 |-- Age Group: string (nullable = true)

In [5]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
capture_account_name = 'spkaccelerjqvse6bhhchxi' # fill in your primary account name
capture_container_name = 'capture' # fill in your container name
capture_relative_path = 'SeattlePublicLibrary/Integrated_Library_System__ILS__Data_Dictionary.csv' # fill in your relative folder path

capture_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (capture_container_name, capture_account_name, capture_relative_path)
print('Primary storage account path: ' + capture_adls_path)

StatementMeta(synasp1, 8, 5, Finished, Available)

Primary storage account path: abfss://capture@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/Integrated_Library_System__ILS__Data_Dictionary.csv

In [6]:
%%pyspark

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
csvSchema = StructType([
  StructField('code', StringType(), True),
  StructField('description', StringType(), True),
  StructField('code_type', StringType(), True), 
  StructField('format_group', StringType(), True),
  StructField('format_subgroup', StringType(), True),
  StructField('category_group', StringType(), True),
  StructField('category_subgroup', StringType(), True),
  StructField('age_group', StringType(), True)   
])

CheckByTPI_capture_df = spark.read.format('csv').option('header', 'True').schema(csvSchema).load(capture_adls_path)

display(CheckByTPI_capture_df.limit(10))



StatementMeta(synasp1, 8, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2f35265a-a395-4096-85c4-d64753a37ab0)

In [7]:
%%pyspark

from pyspark.sql.functions import to_date, to_timestamp, col, date_format, current_timestamp
df_final = (CheckByTPI_capture_df.withColumn('loadDate', date_format(current_timestamp(), 'M/d/y H:m:s a'))
                                 .withColumn("load_date", to_timestamp(col("loadDate"),"M/d/y H:m:s a")).drop("loadDate")
)

StatementMeta(synasp1, 8, 7, Finished, Available)

In [8]:
%%pyspark

# Show Schema
df_final.printSchema()

display(df_final.limit(10))


StatementMeta(synasp1, 8, 8, Finished, Available)

root
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- code_type: string (nullable = true)
 |-- format_group: string (nullable = true)
 |-- format_subgroup: string (nullable = true)
 |-- category_group: string (nullable = true)
 |-- category_subgroup: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- load_date: timestamp (nullable = true)



SynapseWidget(Synapse.DataFrame, 0e550b0e-1493-40e1-956f-e3ff83754c3d)

In [9]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
compose_account_name = 'spkaccelerjqvse6bhhchxi' # fill in your primary account name
compose_container_name = 'compose' # fill in your container name
compose_relative_path = 'SeattlePublicLibrary/IntegratedLibrarySystemILSDataDictionary/' # fill in your relative folder path

compose_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (compose_container_name, compose_account_name, compose_relative_path)
print('Primary storage account path: ' + compose_adls_path)

StatementMeta(synasp1, 8, 9, Finished, Available)

Primary storage account path: abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/IntegratedLibrarySystemILSDataDictionary/

In [10]:
%%pyspark

compose_parquet_path = compose_adls_path + 'datadictionary.parquet'

print('parquet file path: ' + compose_parquet_path)

StatementMeta(synasp1, 8, 10, Finished, Available)

parquet file path: abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/IntegratedLibrarySystemILSDataDictionary/datadictionary.parquet

In [11]:
%%pyspark

df_final.write.parquet(compose_parquet_path, mode = 'overwrite')

StatementMeta(synasp1, 8, 11, Finished, Available)

In [12]:
%%sql

-- Create database SeattlePublicLibrary only if database with same name does not exist
CREATE DATABASE IF NOT EXISTS SeattlePublicLibrary

StatementMeta(synasp1, 8, 12, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
%%sql

-- Create table CheckoutsByTitlePhysicalItemsschemafinal only if table with same name does not exist
CREATE TABLE IF NOT EXISTS SeattlePublicLibrary.integrated_library_system_ils_data_dictionary
(code STRING
 ,description STRING
 ,code_type STRING
 ,format_group STRING
 ,format_subgroup STRING
 ,category_group STRING
 ,category_subgroup STRING
 ,age_group STRING
 ,load_date TIMESTAMP
)
USING PARQUET OPTIONS (path 'abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/IntegratedLibrarySystemILSDataDictionary/datadictionary.parquet')

StatementMeta(synasp1, 8, 13, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>